In [ ]:
import time
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
rental_data_df = pd.read_csv('inputs/RentingOutofFlats2024CSV.csv')

In [ ]:
rental_data_df.head()

In [ ]:
rental_data_df['address'] = rental_data_df['block'] + ' ' + rental_data_df['street_name'] 

In [ ]:
rental_data_df.isna().sum().sum() == 0

In [ ]:
rental_data_df['DATE']       = pd.to_datetime(rental_data_df['rent_approval_date'], errors = 'coerce')
rental_data_df['YEAR']       = rental_data_df['DATE'].dt.year.astype('str')
rental_data_df['MONTH']      = rental_data_df['DATE'].dt.month.astype('str')
rental_data_df['YEAR_MONTH'] = rental_data_df['YEAR'] + '_' + rental_data_df['MONTH']

In [ ]:
rental_data_df.groupby('YEAR').size()

In [ ]:
rental_data_df['town'].nunique()

In [ ]:
rental_data_df.groupby('town').size().sort_values().head()

In [ ]:
rental_data_df.groupby('town').size().sort_values(ascending = False).head()

In [ ]:
rental_data_df.groupby('flat_type').size()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y = rental_data_df['monthly_rent'], x = rental_data_df['town'])
plt.title('Boxplot of Categories', fontsize=16)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))

sns.boxplot(y = rental_data_df['monthly_rent'], x = rental_data_df['town'], hue = rental_data_df['YEAR'])
plt.title('Boxplot of Categories', fontsize=16)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))

sns.boxplot(y = rental_data_df['monthly_rent'], x = rental_data_df['flat_type'], hue = rental_data_df['YEAR'])
plt.title('Boxplot of Categories', fontsize=16)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df = rental_data_df.loc[rental_data_df['town'] == 'TOA PAYOH']

plt.figure(figsize=(20, 8))

sns.boxplot(y = df['monthly_rent'], x = df['flat_type'], hue = df['YEAR'])
plt.title('Boxplot of Categories', fontsize=16)
plt.xlabel('Category', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.xticks(rotation=90)
plt.show()

In [ ]:
import requests
          
dataset_id = "d_473a7a55385ca4a3597bc674a79ffe67"
url = "https://data.gov.sg/api/action/datastore_search?resource_id="  + dataset_id
        
response = requests.get(url)
print(response.json())

In [ ]:
records = response.json()['result']['records']
pd.DataFrame(records)

In [ ]:
for key, value in response.json()['result'].items():
    print(key)

In [ ]:
response.json()

In [ ]:
import os
import json
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [ ]:
load_dotenv()

api_key = os.getenv('API_KEY_26122024')

if api_key:
    print("KEY loaded successfully.")
else:
    print("KEY not found.")


# MRT coordinates

In [ ]:
line_ls = ['EW', 'NS', 'NE', 'CC', 'DT', 'TE', 'BP', 'SE', 'SW', 'PE', 'PW']

station_ls = []

for line in line_ls:
    temp_ls = [line + str(i) for i in range(1, 40)]
    station_ls = station_ls + temp_ls
station_ls

In [ ]:
station_df = pd.DataFrame(station_ls, columns = ['station_code'])
station_df['station_name'] = None
station_df['postal_code']  = None
station_df['latitude']     = None
station_df['longitude']    = None
station_df

In [ ]:
start = time.time()
for station in station_ls:
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={station}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    headers = {"Authorization": api_key}
    response = requests.get(url, headers=headers)
    parsed_data = json.loads(response.text)

    #filter only the MRT station
    for count in range(len(parsed_data['results'])):  
         if ('MRT' in parsed_data['results'][count]['SEARCHVAL']) or ('LRT' in parsed_data['results'][count]['SEARCHVAL']):
             station_df.loc[station_df['station_code'] == station, 'station_name'] = parsed_data['results'][count]['SEARCHVAL']
             station_df.loc[station_df['station_code'] == station, 'postal_code'] = parsed_data['results'][count]['POSTAL']
             station_df.loc[station_df['station_code'] == station, 'latitude'] = parsed_data['results'][count]['LATITUDE']
             station_df.loc[station_df['station_code'] == station, 'longitude'] = parsed_data['results'][count]['LONGITUDE'] 
    
    print(station, start -  time.time())

In [ ]:
station_df[['station_name', 'postal_code', 'latitude', 'longitude']]

In [ ]:
station_df = station_df.drop_duplicates(subset = ['station_name', 'postal_code', 'latitude', 'longitude'], keep='first')
station_df = station_df.loc[station_df['postal_code'] != 'NIL']
station_df = station_df.dropna()